In [1]:
# imports necessary packages
import re
import os
import pandas as pd
from datetime import datetime, timedelta
import statistics as st

# set working directory
os.chdir('D:\Faith and Religion Stuff\Come, Follow Me')

### 18 Sep 2024

As I was studying the scriptures this morning, a few ideas came to me that I wanted to record so I could get back to my study ASAP and finish it before getting engaged in all of this:

* I want to implement a user input aspect to get the start and end dates. I can use Regex to determine whether or not the inputted date is satisfactory for my function.
* This calculator will actually be two calculators - the first will gather information about the start and end reading dates, and determine how many days need to be spent on each talk, and assign dates for each readthrough. **NOTE**: Something I'll need to also account for is whether or not I want to read multiple times. 
* The second calculator will break down each talk into sections of paragraphs based on the number of days that will be spent reading it. 

### 27 Sep 2024

Having dealt with Github desktop giving me problems (it decided out of the blue to stop having access to my D:// drive), with working through the "Dealing with Dates" module twice (once before Github desktop started giving me problems, and then once after I fixed those problems after deleting the work I had already done on the module), I'm now ready to start working on my General Conference calculator. I have:
* the talks from the most recent conference
* a list of information about all the talks
* an understanding of working with dates

I'm hoping that there isn't anything else I'll need to get this project really going now. 

In [2]:
apr_2024_info = pd.read_csv('apr_2024_gc_info.csv')
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146


**NOTE:** Looking the table over, I noticed that my function assigning a day weight of +1 to President Russell M. Nelson's talk did not work properly, so that's something that I adjust for here, and that I will have to fix in my conference calculator later. 

In [3]:
apr_2024_info.at[31,'day_weight'] = 1
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146


One thing I know I'll need to do is establish an initial start date and end date. These are the first day I want to read any portion of any talk from this conference and the last day I want to read any portion of any talk from this conference. These two dates will help me calculate how many days should be spent on each talk, subtracting the ini_start_date from the ini_end_date will tell me how many total days I have to work with. 

In [4]:
ini_start = str(input("What day would you like to start reading?"))
ini_end = str(input("What day would you like to stop reading?"))

ini_start_date = pd.to_datetime(ini_start,format='%m/%d/%Y')
ini_end_date = pd.to_datetime(ini_end,format='%m/%d/%Y')
ini_total_days = ini_end_date - (ini_start_date - timedelta(days=1))
ini_total_days

Timedelta('177 days 00:00:00')

I set the cell above up to take a user input for the start and end dates, to save those, and then to convert those strings to datetime objects. Then I created a new object (ini_total_days) which is a timedelta object that shows the number of days between my start date and end date. In order to ensure that my calculator includes the start date, I had to subtract one day from the initial start date (`ini_start_date`). The resulting `Timedelta` object is 177 days. 

Having that 177 calculation, I can now calculate how many days each talk gets. When I initially did this using a Google Sheet, I did it by first establishing how many times I wanted to read each talk. Then, I established, based on the 177 days I had, I determined how many days would be allotted to each readthrough. From there, I allotted each talk a number of days based on the percentage of total speaking time that talk took to deliver, and added days to talks here and there to make sure there was no remainder of days.

For the sake of consistency, for this calculator I will assume, as I did before, that I will read each talk twice except President Nelson's, which I decided to read 3 times. However, I'll also want to get a user input to establish a read weight column for this calculation. 

In order to account for a possible remainder of days I'll need to establish some sort of standard method for my calculation to follow so I don't have to choose which talks get the remainder days. 

In the cell below, I first establish a read weight (ie, the number of times I want to read each talk) based on role code and organization. Basically, I tell python to look at all of the unique role codes (which represent the organization to which the speaker belongs), then ask the user how many times they want to read talks given by speakers from that organization, and saves that information to a dataframe that then gets merged into the original conference info dataframe in a way that ensures that each talk gets a read weight according to the organization the speaker belongs to. 

The reason I want to include user input when determining the read weight is because I may not want to read every talk more than once in the future, or I may want to read the entire First Presidency's talks more frequently than the others, or something like that. I want the flexibility of being able to specify in the future which organizations' talks I want to read more or less frequently, without having to adjust my code. 

In [5]:
# Initialize an empty read_weights list
read_weights = []

# for every unique role code in the role_code column of the apr_2024_info dataframe...
for item in apr_2024_info.role_code.unique():
    # ... ask the user what the read weight should be and...
    read_weight = int(input(f"How many times would you like to read talks given by the {item}?"))
    # ... save both the role code and the read weight to the read_weights list
    read_weights.append({'role_code':item, 'read_weight':read_weight})

# convert the read_weights list to a dataframe, save with the same name to replace the old item
read_weights = pd.DataFrame(read_weights)

# left-merge the read_weights dataframe to the apr_2024_info dataframe useing the role_code columns as a guide for merging
# left-merge keeps everything in the dataframe being merged to, and only merges data from the second dataframe that has a corresponding value in the original dataframe
apr_2024_info = apr_2024_info.merge(read_weights, on='role_code',how='left')

# display the updated dataframe
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines,read_weight
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151,2
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164,2
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164,2
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181,2
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137,2
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169,2
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179,2
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157,2
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176,2
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146,2


In the cell below, I just calculate and display how many talks, paragraphs, and lines I'll need to read, given that I'll be reading each talk multiple times. The total_time also just calculates how many seconds of speaking I'll be reading. It doesn't make much sense, but I'll use it tomorrow or Monday. 

In [6]:
total_talks = apr_2024_info.Title.nunique()
total_time = sum(apr_2024_info['time'])
total_paragraphs = sum(apr_2024_info['paragraphs'])
total_lines = sum(apr_2024_info['lines'])

print(f'Total talks to read: {total_talks}\n'
      f'Total time: {round(total_time/60)} minutes\n'
      f'Total paragraphs: {total_paragraphs}\n'
      f'Total lines: {total_lines}')

Total talks to read: 32
Total time: 415 minutes
Total paragraphs: 1006
Total lines: 5372


### 30 Sep 2024

Now that I have established how many times I want to read each talk, I can create a measurement of how many days I need to spend on each talk. 

I started by creating a column that gives each talk a proportional "weight" to each talk. This "weight" consists of the average of the following four things:
* 1/32 - There were 32 talks given, and this gives each talk equal weight. 
* The length (in seconds) of the talk divided by the length (in seconds) of the total conference. This gives longer talks more weight. 
* The length (in lines) of the talk divided by the length (in lines) of the total conference. This gives talks with more lines more weight. 
* My personal preference, which I asked for the input on above. In this case, that gives the prophet's talk a little more weight. 

In [7]:
apr_2024_info['conf_cons'] = (
    ((1/32) +                                                                       # Each talk is 1 of 32 given, this treats each equally
     apr_2024_info['time']/total_time +                                             # time weight - longer "heavier"
     apr_2024_info['lines']/total_lines +                                           # lines weight - longer "heavier"
     ((apr_2024_info['day_weight'] + 1)/(apr_2024_info['day_weight'] + 1).sum()))   # preference weight - Prophet's talk "heavier"
     / 4                                                                            # Adding each of those and then dividing by 4 gets the average
)

# Gets the sum of the the conference consumption proportions
# The output of 1 means that 100% of the talk is accounted for, and that no talk(s) are being over- or under-weighted
apr_2024_info.conf_cons.sum()

1.0

After creating that ratio, I created a new column, "num_days, that takes the initial number of days calculation and multiplies that number by the conference consumption column. What this ensures is that each talk is assigned a number of days out of the 177 initial days based on the factors I explained above. To reiterate my rational for considering and averaging each of these weights:
* That every talk is inspired and should be treated as equally of worth
* That some speakers were just given less time to speak (by the direction of the Prophet), and should be given proportional attention
* That some talks are just textually longer than others, and that taking that into account is necessary to give each talk equitable treatment
* The fact that, personally, I just value the talks of some offices (in this case, the Prophet's, but that may change in future conferences) over those of other offices

All of those are balanced against each other, and the resulting balance is a percentage that is multiplied by 177 days to determine the number of days that should be spent on each talk. 

In [8]:
apr_2024_info['tot_num_days'] = ini_total_days.days * apr_2024_info['conf_cons']
apr_2024_info['tot_num_days'] = apr_2024_info['tot_num_days'].round()
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines,read_weight,conf_cons,tot_num_days
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151,2,0.038572,7.0
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164,2,0.022629,4.0
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164,2,0.022237,4.0
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181,2,0.040570,7.0
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137,2,0.021051,4.0
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169,2,0.041255,7.0
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179,2,0.040637,7.0
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157,2,0.021952,4.0
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176,2,0.023117,4.0
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146,2,0.021600,4.0


Subtracting the initial number of days from the sum of all the total days in my dataframe yields a result of 0, meaning that if I study each talk for the specified number of days, I'll finish within the time that I expected. It also means that there are no problems with the math I've done so far. It also addresses a problem I anticipated earlier, which is that of needing to tell my calculator what to do with remaining days. Since there are no remaining days, I don't have to worry about that! 

In [9]:
apr_2024_info.tot_num_days.sum() - ini_total_days.days

0.0

### 3 Oct 2024

Now that I have calculated the number of days to spend on each talk, I now need to work on dividing up the number of days per reading based on the number of readings I want to do and the total number of days I want to read each talk. I think the best approach for that will be to first create a new column for each of the readings I could do. 

In the cell below, I first identify the maximum number of readthroughs of a talk. Then, I set up a loop that creates a new column named "Readthrough # _" for every new column that needs to be made ranging from "Readthrough # 1" through the highest number of readthroughs, and I populate each of those columns with NA values. Finally, I replace all the NA values with zero (so they can be added to later), and then display the dataframe to show that everything worked as I wanted it to. 

In [10]:
apr_2024_info['tot_num_days'] = apr_2024_info['tot_num_days'].astype(int)

In [11]:
# find and save the highest number in the read_weights column
max_reads = apr_2024_info.read_weight.max()

# start a loop that, for every number between 1 and whatever the max_reads number is, inclusive...
for i in range(1,max_reads+1):
    # create a new column of NA values titled "Readthrough # _(whatever number the loop is on)_"
    apr_2024_info[f"Readthrough #{i}"] = pd.NA
    
# convert all NA values to "0"
apr_2024_info.fillna(0, inplace=True)

# Display dataframe to show that everything worked
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines,read_weight,conf_cons,tot_num_days,Readthrough #1,Readthrough #2,Readthrough #3
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151,2,0.038572,7,0,0,0
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164,2,0.022629,4,0,0,0
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164,2,0.022237,4,0,0,0
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181,2,0.040570,7,0,0,0
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137,2,0.021051,4,0,0,0
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169,2,0.041255,7,0,0,0
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179,2,0.040637,7,0,0,0
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157,2,0.021952,4,0,0,0
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176,2,0.023117,4,0,0,0
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146,2,0.021600,4,0,0,0


With those columns for each readthrough made, I could move on to creating and applying a function that would take the total number of days and distribute them equally into each column, with earlier readthroughs being given more days if the number of days could not be split evenly. 

My `distribute_days()` function takes a dataframe like the one I've crafted. At minimum, however, it would need to have a column labeled 'tot_num_days', a column labeled 'read_weight', and an appropriate number of columns labeled 'Readthrough #x', where x is highest number in the 'read_weight' column. A dataframe with any number of rows and additional columns would work, so long as those minimum requirements are met. 

The function iterates over every row of the data. Every time it starts working with a row it looks at the 'tot_num_days' column and saves that value as 'total_days', and it saves 'dist_days' (short for distributed days) as 0. Then, so long as 'dist_days' is less than 'total_days', it adds 1 to one of the Readthrough columns. It does this until 'dist_days' is equal to 'total_days', and then it proceeds to the next row, where it changes the total_days to the corresponding value in that row, and resets distributed days to 0 to start the process over. 

After defining the function, I run it on my entire dataframe, and am left with a dataframe that now has the number of days that should be spent on each readthrough. 

In [12]:
def distribute_days(conf_df):
    """
    This function takes a dataframe like the one I have crafted above and distributes the total number of days into the "Readthrough #_" columns.
    """
    # establish that the function needs to repeat for every row of the dataframe
    for index, row in conf_df.iterrows():
        # get total number of days for that talk
        total_days = row['tot_num_days']
        # initialize number of distributed days as 0
        dist_days = 0
        # establish that the function needs to proceed with the following operation until dist_days and total_days are equal
        while dist_days < total_days:
            # for every whole number between 1 and whatever the read_weight (or number of readthroughs) is...
            for i in range(1,row['read_weight']+1):
                # if dist_days is still less than total_days...
                if dist_days < total_days:
                    # add 1 to whatever value is in the "Readthrough #(number between 1 and number of readthroughs)" column and...
                    conf_df.at[index, f'Readthrough #{i}'] += 1
                    # add 1 to dist_days
                    dist_days += 1
                    # go back to add 1 to the next column until dist_days is no longer less than total_days
                # if/when dist_days is equal to total_days
                else:
                    # break the process of adding one to each column, and move on to the next row to start the process over
                    break
    # when everything is done, the output of this function is the same dataframe with all of the updated columns
    return conf_df

# run the function on my dataframe
distribute_days(apr_2024_info)

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines,read_weight,conf_cons,tot_num_days,Readthrough #1,Readthrough #2,Readthrough #3
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151,2,0.038572,7,4,3,0
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164,2,0.022629,4,2,2,0
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164,2,0.022237,4,2,2,0
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181,2,0.040570,7,4,3,0
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137,2,0.021051,4,2,2,0
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169,2,0.041255,7,4,3,0
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179,2,0.040637,7,4,3,0
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157,2,0.021952,4,2,2,0
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176,2,0.023117,4,2,2,0
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146,2,0.021600,4,2,2,0


Having established how many days needed to be spent on each readthrough, I could then move on to generating which paragraphs of each talk need to be read on each day. This process will be far more complicated. 

In order to assure that each day of reading for each reading of a particular talk takes roughly the same amount of time, I'll be first dividing up the number of lines that need to be read each day, and then I'll find the number of paragraphs that corresponds most closely to that number of lines. 

I'll start by calculating the number of lines that need to be read for each talk, each day, each readthrough. First I need to create a bunch of new columns, somewhat like I did above. In order to do that, I just needed to apply 2 iterative loops so that the cell below looks at the highest number of days for each readthrough and then creates a number of columns for each readthrough that corresponds to the highest number of days. These columns come out being name something like 'r1d1' for readthrough 1 day 1 and r3d3 for readthrough 3 day 3. 

In [13]:
columns = ['pk', 'Author', 'role', 'role_code', 'day_weight', 'Title',
       'Description', 'Link', 'time', 'paragraphs', 'lines', 'read_weight',
       'conf_cons', 'tot_num_days', 'Readthrough #1', 'Readthrough #2',
       'Readthrough #3']

apr_2024_info = apr_2024_info[columns]

In [14]:
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines,read_weight,conf_cons,tot_num_days,Readthrough #1,Readthrough #2,Readthrough #3
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151,2,0.038572,7,4,3,0
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164,2,0.022629,4,2,2,0
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164,2,0.022237,4,2,2,0
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181,2,0.040570,7,4,3,0
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137,2,0.021051,4,2,2,0
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169,2,0.041255,7,4,3,0
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179,2,0.040637,7,4,3,0
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157,2,0.021952,4,2,2,0
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176,2,0.023117,4,2,2,0
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146,2,0.021600,4,2,2,0


I don't know why, but there was no explanation of the contents of the cell below, neither in markdown nor in comments. 

In the cell below, I establish a series of loops to conditionally create a new set of columns in the conference dataframe. 
* the first loop creates a repetition for every number (i) between 1 and whatever the highest number is in the "read_weight" column. The (i) will be used to designate readthrough number. 
* the second loop creates a repetition for every number (x) between 1 and whatever the highest number of days specified in the "Readthrough #i" column. The (x) will be used to designate the day number of any given readthrough.
* the third loop iterates through every row of the apr_2024_info dataframe. 
* the if condition just states defines that if the number of days assigned to any readthrough is zero (because some talks get more readthroughs than others), it should create new columns and fill them with "NA" values. Otherwise, if a readthrough is give a number of days, it should assign the initial number of lines, start point, and end point as 0, to be filled in later. 

In plainer English, the cell below creates columns for storing or defining a start point, a number of lines, and an end point column for every day of every readthrough, regardless of the number of days or readthroughs.

In [15]:


for i in range(1, apr_2024_info['read_weight'].max()+1):
    for x in range(1, apr_2024_info[f'Readthrough #{i}'].max()+1):
        for index, row in apr_2024_info.iterrows():
            if row[f'Readthrough #{i}'] != 0:
                apr_2024_info.at[index, f'r{i}d{x}_start'] = int(0)
                apr_2024_info.at[index, f'r{i}d{x}_lines'] = int(0)
                apr_2024_info.at[index, f'r{i}d{x}_end'] = int(0)
            else:
                apr_2024_info.at[index, f'r{i}d{x}_start'] = pd.NA
                apr_2024_info.at[index, f'r{i}d{x}_lines'] = pd.NA
                apr_2024_info.at[index, f'r{i}d{x}_end'] = pd.NA

apr_2024_info.columns

Index(['pk', 'Author', 'role', 'role_code', 'day_weight', 'Title',
       'Description', 'Link', 'time', 'paragraphs', 'lines', 'read_weight',
       'conf_cons', 'tot_num_days', 'Readthrough #1', 'Readthrough #2',
       'Readthrough #3', 'r1d1_start', 'r1d1_lines', 'r1d1_end', 'r1d2_start',
       'r1d2_lines', 'r1d2_end', 'r1d3_start', 'r1d3_lines', 'r1d3_end',
       'r1d4_start', 'r1d4_lines', 'r1d4_end', 'r2d1_start', 'r2d1_lines',
       'r2d1_end', 'r2d2_start', 'r2d2_lines', 'r2d2_end', 'r2d3_start',
       'r2d3_lines', 'r2d3_end', 'r2d4_start', 'r2d4_lines', 'r2d4_end',
       'r3d1_start', 'r3d1_lines', 'r3d1_end', 'r3d2_start', 'r3d2_lines',
       'r3d2_end', 'r3d3_start', 'r3d3_lines', 'r3d3_end'],
      dtype='object')

Because I found below that some of my columns contained float numbers (numbers with a decimal) rather than integers (whole numbers), I wrote the cell below. I defined a pattern using regular expressions, telling my pattern to look for any non-digit follow by a digit followed by a non-digit followed by any number of digits. Then, I saved the columns that contain float numbers as the object 'float_cols'. I then used the regular expression pattern and function to look through those columns for all the columns that match the pattern I defined. Then I converted only those float columns that matched my pattern into integers. Finally, I display the dtypes to show the datatypes in each column of my dataframe. 

In [28]:
pattern = r'\D\d\D\d+'
float_cols = apr_2024_info.select_dtypes(include=['float'])
change_cols = [col for col in float_cols if re.search(pattern, col)]
apr_2024_info[change_cols] = apr_2024_info[change_cols].astype(int)
apr_2024_info.dtypes

pk                 object
Author             object
role               object
role_code          object
day_weight          int64
Title              object
Description        object
Link               object
time                int64
paragraphs          int64
lines               int64
read_weight         int64
conf_cons         float64
tot_num_days        int32
Readthrough #1      int64
Readthrough #2      int64
Readthrough #3      int64
r1d1_start          int32
r1d1_lines          int32
r1d1_end            int32
r1d2_start          int32
r1d2_lines          int32
r1d2_end            int32
r1d3_start          int32
r1d3_lines          int32
r1d3_end            int32
r1d4_start          int32
r1d4_lines          int32
r1d4_end            int32
r2d1_start          int32
r2d1_lines          int32
r2d1_end            int32
r2d2_start          int32
r2d2_lines          int32
r2d2_end            int32
r2d3_start          int32
r2d3_lines          int32
r2d3_end            int32
r2d4_start  

In [17]:
# failed code - commented out so I can just run all as needed

# def distribute_lines(conf_df):
#     """
#     This function takes a dataframe like the one I have crafted above and distributes the total number of days into the "Readthrough #_" columns.
#     """
#     for index, row in conf_df.iterrows():
#         total_lines = row['lines']
#         dist_lines = 0
#         readthrough = 0

        
#         while dist_lines < total_lines and readthrough <= row['read_weight']:
#             readthrough =+ 1
#             if readthrough > row['read_weight']:
#                 break
#             else:
#                 for i in range(1, row[f'Readthrough #{readthrough}']+1):
#                     if dist_lines < total_lines:
#                         conf_df.at[index, f'r{readthrough}d{i}_lines'] += 1
#                         dist_lines += 1
#                     else:
#                         break
                 

#     return conf_df

# distribute_lines(apr_2024_info)


In [18]:
# failed code - commented out so I can just run all as needed

# def distribute_lines(conf_df):
#     """
#     Distribute the total number of lines into the 'Readthrough #_' columns.
#     """
#     for index, row in conf_df.iterrows():
#         total_lines = row['lines']
#         dist_lines = 0
#         readthrough = 1  # Initialize for the row

#         print(f"Processing row {index}: total_lines = {total_lines}, read_weight = {row['read_weight']}")

#         while dist_lines < total_lines and readthrough <= row['read_weight']:

            

#             for day in range(1, row[f'Readthrough #{readthrough}']+1):
#                 print(f"  Readthrough {readthrough}: max_days_in_readthrough = {row[f'Readthrough #{readthrough}']+1}")
#                 if dist_lines < total_lines:
#                     # Add 1 line to the appropriate day column
#                     conf_df.at[index, f'r{readthrough}d{day}_lines'] += 1
#                     dist_lines += 1
#                     print(f"    Distributed line to r{readthrough}d{day}_lines, dist_lines = {dist_lines}")
#                 else:
#                     print(f"    All lines distributed for this readthrough.")
#                     break  # Stop if we've distributed all the lines
            
#             # After finishing the current readthrough
#             print(f"  Moving to readthrough {readthrough + 1}")
#             readthrough += 1

#         print(f"Finished row {index}: total distributed = {dist_lines}")

#     return conf_df

In [19]:
# failed code - commented out so I can just run all as needed
# distribute_lines(apr_2024_info)

In [20]:
# failed code - commented out so I can just run all as needed
# for i in range(1, apr_2024_info['read_weight'].max()+1):
#     for x in range(1, apr_2024_info[f'Readthrough #{i}'].max()+1):
#         for index, row in apr_2024_info.iterrows():
#             if row[f'Readthrough #{i}'] != 0:
#                 apr_2024_info.at[index, f'r{i}_lines_per_day'] = round(row['lines']/row[f'Readthrough #{i}'])
#             else:
#                 apr_2024_info.at[index, f'r{i}_lines_per_day'] = 0
#         for index, row in apr_2024_info.iterrows():
#             if row[f'r{i}_lines_per_day'] != 0:
#                 if i == 1:
#                     apr_2024_info.at[index, f'r{i}d{x}_lines'] = f"1 - {row[f'r{i}_lines_per_day']}"
#                 elif (x-1)*(row[f'r{i}_lines_per_day']) <= row['lines']:
#                     apr_2024_info.at[index, f'r{i}d{x}_lines'] = f"{(x-1)*(row[f'r{i}_lines_per_day'])} - {x*(row[f'r{i}_lines_per_day'])}"
#                 else:
#                     apr_2024_info.at[index, f'r{i}d{x}_lines'] = 0
            
# apr_2024_info

Having those columns made, I can now fill them with the number of lines. 

Okay, I'm out of time for today. This has been a beast. 

### 4 Oct 2024

Picking up where I left off yesterday, I am currently working on trying to distribute the number of lines across the days in each readthrough. Once those lines have been distributed, I can then incorporate the tables I made that have the text, lines, etc, of each talk. 

In [21]:
def distribute_lines(conf_df):
    """
    This function takes a dataframe like the one I have crafted above and distributes the total number of days into the "Readthrough #_" columns.
    """
    # initialize readthrough count as zero
    readthrough = 0
    # establish maximum number of readthroughs so the loop I create below knows when to end or stop
    max_readthroughs = conf_df['read_weight'].max()
    # start a loop of action that will continue until readthroughs is bigger than max_readthroughs, at which point it will stop
    while readthrough <= max_readthroughs:
        # A - first action: add 1 to readthroughs, establishing which readthrough we are working with
        readthrough += 1
        # B - Check if the current readthrough exists in the DataFrame
        if f'Readthrough #{readthrough}' not in conf_df.columns:
            break  # If the column doesn't exist, exit the loop

        # C - second action: for every row in the dataframe do the following:
        for index, row in conf_df.iterrows():
            # D - second action, cont'd: save the info in the row 'lines' as the total number of lines to be distributed
            total_lines = row['lines']
            # E - second action, cont'd: initialize count of distributed lines as zero
            dist_lines = 0
            # debugging print line - remove later
            print(f"Row {index} - Readthrough {readthrough} has {row[f'Readthrough #{readthrough}']} days.")
            # F - check whether the number of days assigned to a talk in a particular readthrough is 0
            if row[f'Readthrough #{readthrough}'] == 0:
                # debugging print line - remove later
                print(f"Skipping row {index} - Readthrough {readthrough} because it has 0 days.")
                # if that talk has zero days alloted for that readthrough, skip to the next talk
                continue
            
            # G - second action, cont'd: open another while loop that will continue until the number of distributed lines is equal to the number of total lines
            while dist_lines < total_lines:
                # H - first action of second loop: open another loop that operates i number of times, where i is the number of days in the readthrough
                for i in range(1, row[f'Readthrough #{readthrough}']+1):
                    # debugging print line - remove later
                    print(f"Processing row {index}, readthrough {readthrough}, day {i}.")
                    # I - first action of second loop, cont'd: add 1 to the corresponding i day of the current readthrough
                    conf_df.at[index, f'r{readthrough}d{i}_lines'] += 1
                    # J - first action of second loop, cont'd: add 1 to the number of distributed lines
                    dist_lines += 1
                    # debugging print line - remove later
                    print(f"Distributed lines: {dist_lines}/{total_lines}")
                    # K - check if dist_lines is less than total_lines: 
                    if dist_lines >= total_lines:
                        break
                        # Unstated action: If it is less, return to point H and repeat this loop.
                        # If not, end *this* loop and return to point D for the next row.
                
        # L - check if readthroughs is less than or equal to max_readthroughs
        if readthrough > max_readthroughs:
            # If it is, repeat this loop, starting with point A
            break
            # Unstated action: If not, end this loop and go to point M.
    
    # M - return the newly modified dataframe
    return conf_df


In [22]:
distribute_lines(apr_2024_info)

Row 0 - Readthrough 1 has 4 days.
Processing row 0, readthrough 1, day 1.
Distributed lines: 1/151
Processing row 0, readthrough 1, day 2.
Distributed lines: 2/151
Processing row 0, readthrough 1, day 3.
Distributed lines: 3/151
Processing row 0, readthrough 1, day 4.
Distributed lines: 4/151
Processing row 0, readthrough 1, day 1.
Distributed lines: 5/151
Processing row 0, readthrough 1, day 2.
Distributed lines: 6/151
Processing row 0, readthrough 1, day 3.
Distributed lines: 7/151
Processing row 0, readthrough 1, day 4.
Distributed lines: 8/151
Processing row 0, readthrough 1, day 1.
Distributed lines: 9/151
Processing row 0, readthrough 1, day 2.
Distributed lines: 10/151
Processing row 0, readthrough 1, day 3.
Distributed lines: 11/151
Processing row 0, readthrough 1, day 4.
Distributed lines: 12/151
Processing row 0, readthrough 1, day 1.
Distributed lines: 13/151
Processing row 0, readthrough 1, day 2.
Distributed lines: 14/151
Processing row 0, readthrough 1, day 3.
Distributed

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,...,r2d4_end,r3d1_start,r3d1_lines,r3d1_end,r3d2_start,r3d2_lines,r3d2_end,r3d3_start,r3d3_lines,r3d3_end
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,...,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,...,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,...,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,...,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,...,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,...,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,...,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,...,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,...,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,...,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


Okay, I finally got that all worked out. The function above basically just makes sure that each day of each readthrough of each talk has a (mostly) equal whole number of lines to be read. From here, over the next few days, I'll need to convert those calculations into the lines at which that day is supposed to end (as the previous day's end line will be the current day's start line, unless it's the first day, in which case the start line will be 1). That's work for another day, though, as I've got work I need to get to.

### 5 Oct 2024

Having succeeded yesterday in getting the number of lines to be read each day of each readthrough of each talk, now I can convert those into endpoints. 

In order to do that in a way that will be applicable no matter the number of readthroughs or days that are spent on each readthrough in future conferences, I need to make use of regular expressions. I will convert the operations below into a function later, but first I'll explain what's happening. 

First, I save the column names in my conference dataframe as a list. 
Then, I establish a pattern for identifying columns that containing the number of verses to be read on the first day of any readthrough, and a pattern for identifying columns containing the number of verses to be read on all other days of any readthrough. 
Next, I use Python's built in list comprehension to create a separate list of each of those columns.
Finally, I employ a conditional statement that prints each list so long as the list exists. (The list will only not exist if no matches are found, which would only happen if there was an error at some previous point of my code.)

In [23]:
# save column names as a list
col_names = list(apr_2024_info.columns)

# establish regular expression patterns
d1_pattern = r'\D\d+\D1'
non_d1_pattern = r'\D\d+\D[2-9]+'

# use list comprehension to create lists of column names matching either pattern
d1_list = [col for col in col_names if re.search(d1_pattern, col)]
non_d1_list = [col for col in col_names if re.search(non_d1_pattern, col)]

# If a list of columns matching the day 1 expression exists, display it
if d1_list:
    print(f'D1 Matches found: {d1_list}')
# Otherwise, state that no matches were found. 
else:
    print('No D1 matches found.')

# If a list of columns matching the non-day 1 expression exists, display it
if non_d1_list:
    print(f'Non-d1 Matches found: {non_d1_list}')
# Otherwise, state that no matches were found. 
else:
    print('No non-d1 matches found.')

D1 Matches found: ['r1d1_start', 'r1d1_lines', 'r1d1_end', 'r2d1_start', 'r2d1_lines', 'r2d1_end', 'r3d1_start', 'r3d1_lines', 'r3d1_end']
Non-d1 Matches found: ['r1d2_start', 'r1d2_lines', 'r1d2_end', 'r1d3_start', 'r1d3_lines', 'r1d3_end', 'r1d4_start', 'r1d4_lines', 'r1d4_end', 'r2d2_start', 'r2d2_lines', 'r2d2_end', 'r2d3_start', 'r2d3_lines', 'r2d3_end', 'r2d4_start', 'r2d4_lines', 'r2d4_end', 'r3d2_start', 'r3d2_lines', 'r3d2_end', 'r3d3_start', 'r3d3_lines', 'r3d3_end']


The output of the cell above shows that d1_list properly contains a list of all of the r_d1 columns, while the non_d1_list properly contains a list of all of the other r_d_ columns, meaning that it worked exactly the way I wanted it to. 

The reason I need these is so that I can perform specific operations on any number specifically targetted columns, where the targetting is done by finding the names using regular expressions. 

This, unfortunately, was all I had time for today, but I'm happy with what I accomplished in a small amount of time. 

### 6 Oct 2024

Having figured out that I *can* identify columns by what day of any given readthrough they contain information for, I need to figure out how to set up my Regex scan for columns so that it will not require input from me in the future. 

My end goal for this stage of the calculation is to have a set of columns that define the starting and ending line of each day of each readthrough. I can use those starting and ending points for ease of calculating the number of paragraphs to read. 

Simply enough, for all the day ones of each readthrough, I know the starting point is line 1, and I know the ending point is the number of lines to be read that day. I also know that the starting point for every day 2 is the same as the ending point for day 1. The end of day 2, though, and everything after that, gets more complicated. So, I need to create a column for the day 2 end point, and for the start and end point of every day thereafter. I also need to automate that so that my calculator works irrespective of the number of days assigned to each readthrough, which has been the most complicated part of this whole process. 

I went back to the cell above where I created a series of new columns for each day of each readthrough, and added some code to also create, at the same time, empty columns for the start and end points of each readthrough. Tomorrow or the next day, I can populate those start and end points, and then I can get to the next step of my calculator. 

IndentationError: expected an indented block after 'for' statement on line 16 (472407229.py, line 22)